In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
train_comments = train['comment_text'].fillna("_na_").values
print train_comments[8]

Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169


In [5]:
train_y = train[ ['toxic' , 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate'] ].values
train_y[12]

array([1, 0, 0, 0, 0, 0])

In [6]:
max_words = 30000
max_len = 100

In [7]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_comments)
train_comments_tokenized = tokenizer.texts_to_sequences(train_comments)
print train_comments_tokenized[8]

[263, 22, 1, 286, 24, 1476, 2, 6, 489, 73, 14, 9215, 2, 330, 215, 10, 1, 23, 1473, 51, 47, 2898, 15, 35, 12, 199, 73, 1051, 1060, 9, 11, 16, 57, 1390, 37, 48, 40, 81, 11, 12, 480, 17, 5, 273, 7, 18, 55, 2, 1, 4850, 8153, 27, 26, 78, 669, 5, 1368, 11, 687, 2, 53, 8, 5, 702, 7774, 23, 9, 1094, 6, 46, 468, 41, 251, 16, 159, 324, 21, 3566, 10, 13823, 3439, 4584, 3890, 4851]


In [8]:
train_X = pad_sequences(train_comments_tokenized, maxlen=max_len, padding='post')
train_X[8]

array([  263,    22,     1,   286,    24,  1476,     2,     6,   489,
          73,    14,  9215,     2,   330,   215,    10,     1,    23,
        1473,    51,    47,  2898,    15,    35,    12,   199,    73,
        1051,  1060,     9,    11,    16,    57,  1390,    37,    48,
          40,    81,    11,    12,   480,    17,     5,   273,     7,
          18,    55,     2,     1,  4850,  8153,    27,    26,    78,
         669,     5,  1368,    11,   687,     2,    53,     8,     5,
         702,  7774,    23,     9,  1094,     6,    46,   468,    41,
         251,    16,   159,   324,    21,  3566,    10, 13823,  3439,
        4584,  3890,  4851,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)

In [9]:
train_X.shape

(159571, 100)

In [18]:
def BiLSTM():
    X_inp = Input( shape=(max_len,) )
    
    X = Embedding(max_words, 128)(X_inp)
    
    X = Bidirectional(LSTM(200, return_sequences=True))(X)
    
    X = Dropout(0.1)(X)
    
    X = LSTM(100, return_sequences=True)(X)
    
    X = GlobalAvgPool1D()(X)
    
    X = Dense(50, activation='relu')(X)
    
    X = Dropout(0.1)(X)
    
    X = Dense(6, activation='sigmoid')(X)
    
    model  = Model(inputs=X_inp, outputs=X)
    
    return model
    
    

In [19]:
toxicity_model1 = BiLSTM()

In [20]:

toxicity_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
toxicity_model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 128)          3840000   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100, 400)          526400    
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 400)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100, 100)          200400    
_________________________________________________________________
global_average_pooling1d_5 ( (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
__________

In [22]:
batch_size = 64
epochs = 2
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
toxicity_model1.fit(train_X,train_y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[es])

Train on 127656 samples, validate on 31915 samples
Epoch 1/2
127656/127656 [==============================] - 2584s 20ms/step - loss: 0.1026 - acc: 0.9705 - val_loss: 0.0573 - val_acc: 0.9808
Epoch 2/2
127656/127656 [==============================] - 2575s 20ms/step - loss: 0.0496 - acc: 0.9821 - val_loss: 0.0510 - val_acc: 0.9819


In [23]:
def predict(inp, model):
    inp_tokenized = tokenizer.texts_to_sequences([inp])
    final_inp = pad_sequences(inp_tokenized, maxlen=max_len, padding='post')
    print model.predict(final_inp)
    

In [24]:
predict("I Love You", toxicity_model1)

[[0.1383738  0.00014419 0.00933317 0.00054649 0.01502659 0.00263758]]


In [25]:
predict("I Hate You", toxicity_model1)

[[0.71565217 0.00505066 0.13578227 0.00840953 0.22371264 0.02951044]]


In [26]:
predict("You are an idiot", toxicity_model1)

[[0.9422976  0.02767189 0.6440113  0.01915897 0.52128506 0.06754439]]


In [27]:
predict("You are a moron", toxicity_model1)

[[0.9208554  0.02300669 0.530721   0.0194425  0.47277844 0.06537705]]


In [28]:
toxicity_model1.save_weights("toxicity_model2_weights.h5")

In [29]:
import pickle

with open('tokenizer.pk', 'wb') as tk:
    pickle.dump(tokenizer, tk, protocol=pickle.HIGHEST_PROTOCOL)
